In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np

from tensorflow import keras
from keras import Input, Sequential
from keras.layers import AveragePooling2D, Conv2D, Dense, Flatten

In [ ]:
# Download and import custom Subsampling layer.
!curl -sO https://raw.githubusercontent.com/mbrukman/reimplementing-ml-papers/main/lenet/subsampling.py
from subsampling import Subsampling

In [ ]:
# Load the MNIST dataset.
(x_train_raw, y_train_raw), (x_test_raw, y_test_raw) = keras.datasets.mnist.load_data()

In [ ]:
# Examine the dataset shape.
print("Raw data:")
print(f"Train x: {x_train_raw.shape}")
print(f"      y: {y_train_raw.shape}")
print(f"Test  x: {x_test_raw.shape}")
print(f"      y: {y_test_raw.shape}")

Raw data:
Train x: (60000, 28, 28)
      y: (60000,)
Test  x: (10000, 28, 28)
      y: (10000,)


In [ ]:
num_classes = 10

# Expand the dimensions of our inputs to ensure their shape is (28, 28, 1),
# rather than (28, 28) which is what they are by default.
x_train = np.expand_dims(x_train_raw, -1)
x_test = np.expand_dims(x_test_raw, -1)

# Scale train and test inputs by converting them from range of [0, 255] to [0.0, 1.0]
x_train = x_train_raw.astype('float32') / 255.0
x_test = x_test_raw.astype('float32') / 255.0

# Convert the output to categorical one-hot encoding to match the output of our
# network.
y_train = keras.utils.to_categorical(y_train_raw, num_classes)
y_test = keras.utils.to_categorical(y_test_raw, num_classes)

print("\nProcessed data:")
print(f"Train x: {x_train.shape}")
print(f"      y: {y_train.shape}")
print(f"Test  x: {x_test.shape}")
print(f"      y: {y_test.shape}")


Processed data:
Train x: (60000, 28, 28)
      y: (60000, 10)
Test  x: (10000, 28, 28)
      y: (10000, 10)


In [ ]:
# Examine the format of the Y values before and after preprocessing.
print('Y values before preprocessing:')
print(y_train_raw[0:4])

print('\nY values after preprocessing:')
print(y_train[0:4])

Y values before preprocessing:
[5 0 4 1]

Y values after preprocessing:
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
def lenet_activation(a: float) -> float:
    A = 1.7159
    S = 2. / 3
    return A * keras.activations.tanh(S * a)

In [ ]:
model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation=lenet_activation, name="C1"),
    Subsampling(pool_size=(2, 2), strides=(2, 2), activation=lenet_activation, name="S2"),
    Conv2D(filters=16, kernel_size=(5, 5), activation=lenet_activation, name="C3"),
    Subsampling(pool_size=(2, 2), strides=(2, 2), activation=lenet_activation, name="S4"),
    Flatten(),
    Dense(120, activation=lenet_activation, name="C5"),
    Dense(84, activation=lenet_activation, name="F6"),
    Dense(10, activation='softmax', name="Output"),
], name="LeNet-5")

model.summary()

Model: "LeNet-5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C1 (Conv2D)                 (None, 28, 28, 6)         156       
                                                                 
 S2 (Subsampling)            (None, 14, 14, 6)         12        
                                                                 
 C3 (Conv2D)                 (None, 10, 10, 16)        2416      
                                                                 
 S4 (Subsampling)            (None, 5, 5, 16)          32        
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 C5 (Dense)                  (None, 120)               48120     
                                                                 
 F6 (Dense)                  (None, 84)                1016

In [ ]:
# Compile the model with optimizer and loss function.
opt = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
1875/1875 [==============================] - 18s 6ms/step - loss: 0.2624 - accuracy: 0.9194
Epoch 2/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0950 - accuracy: 0.9710
Epoch 3/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0619 - accuracy: 0.9808
Epoch 4/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0489 - accuracy: 0.9847
Epoch 5/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0386 - accuracy: 0.9876
Epoch 6/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0334 - accuracy: 0.9893
Epoch 7/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0268 - accuracy: 0.9913
Epoch 8/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0232 - accuracy: 0.9925
Epoch 9/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0197 - accuracy: 0.9935
Epoch 10/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.01

In [ ]:
# Evaluate the model
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0566 - accuracy: 0.9860


[0.05663033574819565, 0.9860000014305115]